In [1]:
import numpy as np
from tqdm import tqdm_notebook as tqdm

import numpy as np

In [ ]:
# Base code : https://yamalab.tistory.com/92

class MatrixFactorization():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix      # 평점행렬
        :param k: latent parameter   # user latent와 item latent의 차원 수
        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update
        :param epochs: training epochs
        :param verbose: print status
        """
        self._R = R
        self._num_users, self._num_items = R.shape   # 유저의 수와 아이템 수
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose


    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        참고: self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """

        # init latent features
        self._P = np.random.normal(size=(self._num_users, self._k))   # user에 대한 latent space
        self._Q = np.random.normal(size=(self._num_items, self._k))   # item에 대한 latent space

        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)])

        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):    # for문을 돌면서 좋은 값으로 수렴하게 됨
            # rating이 존재하는 index를 기준으로 training
            xi, yi = self._R.nonzero()  # 0이 아닌 부분의 x y 값을 출력
            for i, j in zip(xi, yi):
                self.gradient_descent(i, j, self._R[i, j])
            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))


    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        # 참고: http://codepractice.tistory.com/90
        xi, yi = self._R.nonzero()
        # predicted = self.get_complete_matrix()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - self.get_prediction(x, y), 2)
        return np.sqrt(cost/len(xi))


    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """

        dp = (error * self._Q[j, :]) - (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) - (self._reg_param * self._Q[j, :])
        return dp, dq



    def gradient_descent(self, i, j, rating):   
        """
        graident descent function

        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """

        # get error
        prediction = self.get_prediction(i, j)  # 행렬의 원소 위치와 평점 값을 받아서 각 Latent Matrix 곱을 통해 행렬 생성한 prediction을 가져옴
        error = rating - prediction

        # update biases
        self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])

        # update latent feature
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] += self._learning_rate * dp
        self._Q[j, :] += self._learning_rate * dq


    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)


    def get_complete_matrix(self):
        """
        computer complete matrix PXQ + P.bias + Q.bias + global bias

        - PXQ 행렬에 b_P[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_Q[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)

In [2]:
# run example
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5)
    R = np.array([
        [1, 0, 0, 1, 3],
        [2, 0, 3, 1, 1],
        [1, 2, 0, 5, 0],
        [1, 0, 0, 4, 4],
        [2, 1, 5, 4, 0],
        [5, 1, 5, 4, 0],
        [0, 0, 0, 1, 0],
    ])

    # P, Q is (7 X k), (k X 5) matrix
    
# rating matrix를 input으로 받음
# 위 매트릭스의 경우 사용자가 아이템에 매긴 평점으로 이루어져 있음

In [3]:
%%time
factorizer = MatrixFactorization(R, k=3, learning_rate=0.01, reg_param=0.01, epochs=100, verbose=True)
factorizer.fit()

Iteration: 10 ; cost = 0.9730
Iteration: 20 ; cost = 0.6849
Iteration: 30 ; cost = 0.5249
Iteration: 40 ; cost = 0.4164
Iteration: 50 ; cost = 0.3367
Iteration: 60 ; cost = 0.2757
Iteration: 70 ; cost = 0.2284
Iteration: 80 ; cost = 0.1915
Iteration: 90 ; cost = 0.1627
Iteration: 100 ; cost = 0.1401
Wall time: 68.1 ms


In [4]:
factorizer.get_complete_matrix()

array([[1.03314714, 4.07792927, 3.14985413, 0.99680343, 2.98841942],
       [1.67515468, 2.0101343 , 2.72124777, 1.35286132, 1.04808917],
       [0.86826498, 1.98528827, 6.68620748, 5.0329913 , 4.29737045],
       [1.20813813, 3.01474644, 5.68830816, 3.87970095, 4.0025781 ],
       [2.13558229, 1.03335595, 5.0688246 , 3.88835299, 2.78124103],
       [4.95499072, 1.00474342, 4.95576336, 4.04153736, 3.98243287],
       [2.79577574, 2.13281428, 2.43646489, 0.96171412, 2.17780753]])